In [ ]:
using Dates
# using Revise
import PlotlyJS

include("../Sensors/Sensors.jl")
include("../PreProcessing/PreProcessing.jl")
include("../Utils/Helpers.jl")
include("../Utils/Visualizations.jl")
using .Sensors, .PreProcessing, .Visualizations, .Helpers;

## Load the data and format it

In [ ]:
#= datafolder = "../../Data/Twitter/"
datafile = datafolder * "all_links_processed.json"
data = load_json(datafile) =#

datafolder = "../../Data/Twitter/COP26_processed_lightweight/"
datafiles = [file for file in readdir(datafolder) if occursin(".json", file)]
frames = [load_json(datafolder * file) for file in datafiles]
data = vcat(frames...)

data = data[.~ismissing.(data."domain"), :]

to_datetime = x -> DateTime(split(x, '.')[1], "yyyy-mm-ddTHH:MM:SS")
data."created_at" = to_datetime.(data."created_at");

## Define the partition, actors and actions

In [ ]:
part_fun = cop_26_dates
actor_fun = follower_count
action_fun = trust_score

df = data |> part_fun |> action_fun |> actor_fun

actions = sort(unique(df.action))
actors = sort(unique(df.actor))
partitions = sort(unique(df.partition));

## Statistics on the data

In [ ]:
show(plot_actor_frequency(df, save=false, filename="../../Results/actor_frequency.pdf"))

In [ ]:
plot_actor_wordcloud(df, Nactor=300, save=false, filename="../../Results/actor_wordcloud.svg")

In [ ]:
show(plot_action_frequency(df, save=false, filename="../../Results/action_frequency.pdf"))

## Compute influence graph

In [ ]:
hours = 2
minutes = 0;

In [ ]:
total_min = hours*60 + minutes

tsg = TimeSeriesGenerator(Minute(total_min), standardize=false)

ig = InfluenceGraphGenerator()
# ig = InfluenceGraphGenerator(SMeasure)
# ig = InfluenceGraphGenerator(JointDistanceDistribution, Nsurro=100, alpha=0.001, seed=1234)
# ig = InfluenceGraphGenerator(TransferEntropy)

cuttoff = 0.5
# icg = InfluenceCascadeGenerator(cuttoff)
icg = InfluenceCascadeGenerator(WithoutCuttoff)
# icg = InfluenceCascadeGenerator(cuttoff)

In [ ]:
time_series = observe(df, tsg)
influence_graphs = observe(time_series, ig);

## Compute influence cascade

In [ ]:
influence_cascades = observe.(influence_graphs, Ref(icg))

edge_types = [string(n1, " to ", n2) for n1 in actions for n2 in actions]
push!(edge_types, "Any Edge");

## Plot influence graph

In [ ]:
partition = "During COP26"
edge = "Any Edge";

In [ ]:
simplifier = make_simplifier(edge, cuttoff, edge_types)
partition_index = (1:length(partitions))[findfirst(partition .== partitions)]
	
# In this case we plot the graph on a world map
if actor_fun == country
	PlotlyJS.plot(map_plot(df)...)
# In this case we plot a simple graph of the actors
else
	plot_graph(influence_graphs[partition_index], df, simplifier)
end

## Plot cascades and statistics

In [ ]:
partition_index = 3
influencer_indices = [ic.root for ic in influence_cascades[partition_index]]
influencers = sort(actors[influencer_indices])

In [ ]:
influencer_node1 = influencers[4]

PlotlyJS.plot(plot_cascade_sankey(influence_cascades[partition_index][findfirst(influencer_node1 .== influencers)], df)...)
# PlotlyJS.savefig(PlotlyJS.plot(plot_cascade_sankey(influence_cascades[partition_index][findfirst(influencer_node1 .== influencers)], df)...), "../../Results/cascade.pdf")

In [ ]:
show(plot_actors_per_level(influence_cascades, df, log=false))

In [ ]:
show(plot_actors_per_level(influence_cascades, df, save=false, filename="../../Results/mean_actor_cascade.pdf", log=true, inner_spacing=0.02, reorder=[2, 3, 1]))